<a href="https://colab.research.google.com/github/AISpaceXDragon/IIIT_Training/blob/main/Copy_of_FineTuningBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'daigt-proper-train-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3942644%2F6890527%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240304%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240304T101059Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da504951d6a1d0b970e9f5d3b38f9c92c505af34ac887721217d8e314410fbae6bc16ee721f5dfca1f7f002ba36a9526a9aeff781cef3ec75a4210b73ce40549f604559c8caaab33fa6fae460ed8cd7f417b57ea3716bcdcea7a2e06164f43b1dda469bc748157ea019f4161707ae502cfd0394cd8466ca2bde40d56762512b1ef89e55ef6d47e7b813d0be8ca9cf866fcb8210b6557e4c7182326d71fbf7a83b491bb306d39225b9c43195d168be2b7d58acf2e7fa44b8163d90590b6eb871a1fa74fcd416e6921e5edd5f2975a3a3f75363d9a0f86361e4f995e023d0edd34cade4bc796d57689b8b61a41591bde53fab2e11217ab14e1f858b3e9313fddac7,daigt-v4-train-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4309752%2F7409832%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240304%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240304T101059Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4f4f68629d4f2969d6c00531349d00f8d5b508ad750e7e32e3912effa26de1909bdff58c64690262867eeecb4dacccb4acbef1b4bc705f0a6ad1dd388743cd2cc40bbd5b77abf623234942ade8e4614b91e6acfca18515b46d64f9a3120c73b28e8d1b27dd466fbe4ffcaae2738dceb3ad6afe1442239cb68d7a7432248d9ba8fef5cb06b4f17c9fdd5172172418ba6413af0f2449bde17e37ecbd05dbbcb965c01ef61f422a6636c2f08ab303f8924239880b29943d58877082f6cbb8169652450c5da61e0be4850ac22715f1b2023915df5baf6a1301fa0876c6ab62462a5fc28744783b70c326f2ebc9ca37386340080d42a7e6b8f12b02adf23c343be31e,outputftllm:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4519900%2F7734462%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240304%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240304T101059Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db503d8dbf40a438084307325b62b416bc1ba8d96bba50d1ba014d49a3d3f23df6338c22f69354d3d169abc3e40391224ec0aac9ebe7443f868d90cef13b289844d57df37b0e03be95d52d036c2ac7d3074796a45fbe3b44e5d3584ca7fadbeb50728b82676026789341fa1a3a0fe95ef8fd106f503351a1b57e08e8f5cdfc63cfbd0ebcfcef8dae79deee4d67e0a04b979752fc70565565cf20e93b98b7d6c102c5c388c8939e06196c4c097484592ef051b491d90aa1d077ab8d19a37e1b85b34d47f0d05f3badc000fa81bf5b5284db4411dca6b79296f2265cde969b550412cb94c08d117eca33350efd9001c3d912c13fca86b4b47801c2f36f84a7e4316'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 124388618 bytes downloaded
Downloaded and uncompressed: daigt-proper-train-dataset
[==================================================] 51270920 bytes downloaded
Downloaded and uncompressed: daigt-v4-train-dataset
[==================================================] 2322265730 bytes downloaded
Downloaded and uncompressed: outputftllm
Data source import complete.


#**Accessing the "State of the Art Generative models" and generating respones to queries, Detection of AI Generated Text by Fine Tuning LLM(BERT)**

In this project I have explained how to access LLM "Gemini",by Google with the use of API, and I have also shown the process to fine tune a LLM i.e., "BERT" in here, for the purpose of **"Detection of AI Generated Text"**.

To access State of the Art LLM model "Gemini" we have two methods

1.   Use it on https://gemini.google.com/
2.   If you are a developer, you can use the API key to access it.

The procedure to access Gemini by API is given below:

1.   Visit https://aistudio.google.com/app/apikey and click on Sign up(assuming you have a Gmail account).
2.   On the top you can click on "Create API Key" and use it!





In [1]:
#@title Configure Gemini API key

# access your Gemini API key

import google.generativeai as genai
from google.colab import userdata

gemini_api_secret_name = 'MyApi'  # @param {type: "string"}

try:
  GOOGLE_API_KEY=userdata.get(gemini_api_secret_name)
  genai.configure(api_key=GOOGLE_API_KEY)
except userdata.SecretNotFoundError as e:
   print(f'''Secret not found\n\nThis expects you to create a secret named {gemini_api_secret_name} in Colab\n\nVisit https://makersuite.google.com/app/apikey to create an API key\n\nStore that in the secrets section on the left side of the notebook (key icon)\n\nName the secret {gemini_api_secret_name}''')
   raise e
except userdata.NotebookAccessError as e:
  print(f'''You need to grant this notebook access to the {gemini_api_secret_name} secret in order for the notebook to access Gemini on your behalf.''')
  raise e
except Exception as e:
  # unknown error
  print(f"There was an unknown error. Ensure you have a secret {gemini_api_secret_name} stored in Colab and it's a valid key from https://makersuite.google.com/app/apikey")
  raise e

model = genai.GenerativeModel('gemini-pro')

In [2]:
#@title Ask a question!

text = 'Write a python function that calculates the distance between any two latitudes and longitudes on earth' # @param {type:"string"}

# ask Gemini
model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat(history=[])

response = chat.send_message('%s -- Please answer as concisely as you can, avoiding any extra conversation or text' % text)
gemini_response = response.text


# render the diff

# importing these every execution is unnecessary but avoids another notebook cell
from IPython.display import HTML
import difflib

# omit the legend to slim down the UI
difflib.HtmlDiff._legend = ''

HTML(difflib.HtmlDiff().make_file(gemini_response.splitlines(), 'Gemini'))

t,1,```python,t,1,G
,2,"from math import acos, sin, cos",,2,e
,3,,,3,m
,4,"def distance(lat1, lon1, lat2, lon2):",,4,i
,5,p = 0.017453292519943295,,5,n
,6,a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2,,6,i
,7,return 12742 * acos(a),,,
,8,```,,,


In [2]:
#@title Downloading all the necessary packages required for the implementation
!pip install transformers transformers[torch] tqdm kaggle datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00


In [3]:
#@title Importing all the necessary packages and methods.
import pandas as pd
import numpy as np
import torch
import tqdm
import transformers
from datasets import *
import evaluate
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding

In [7]:
#@title Loading the DAIGT dataset to train the BERT for classification of text.
train_data = pd.read_csv("/kaggle/input/daigt-v4-train-dataset/train_v4_drcat_01.csv")
train_data

,text,label,prompt_name,source,RDizzl3_seven,model
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False,human
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False,human
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False,human
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False,human
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False,human
...,...,...,...,...,...,...
73568,I am writing you today to disagree with your t...,1,Grades for extracurricular activities,persuade_finetuned_llamas,False,llama
73569,"Dear Principal,\n\nIn conclusion, I would obse...",1,Grades for extracurricular activities,persuade_finetuned_llamas,False,llama
73570,"Dear Mrs. Principal,\n\nin these kinds of cons...",1,Grades for extracurricular activities,persuade_finetuned_llamas,False,llama
73571,I enjoyed Form five and excitedly ex claims ed...,1,Grades for extracurricular activities,persuade_finetuned_llamas,False,llama


In [8]:
copy_data = train_data
copy_data

,text,label,prompt_name,source,RDizzl3_seven,model
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False,human
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False,human
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False,human
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False,human
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False,human
...,...,...,...,...,...,...
73568,I am writing you today to disagree with your t...,1,Grades for extracurricular activities,persuade_finetuned_llamas,False,llama
73569,"Dear Principal,\n\nIn conclusion, I would obse...",1,Grades for extracurricular activities,persuade_finetuned_llamas,False,llama
73570,"Dear Mrs. Principal,\n\nin these kinds of cons...",1,Grades for extracurricular activities,persuade_finetuned_llamas,False,llama
73571,I enjoyed Form five and excitedly ex claims ed...,1,Grades for extracurricular activities,persuade_finetuned_llamas,False,llama


We see that the number of non human data points are more in number.Training on such a dataset may lead to a biased model.So we must balance the dataset.

In [9]:
#@title Checking the distribution of the dataset.
print(train_data["model"].unique())
print((train_data["model"].to_numpy() == "human").sum())
print((train_data["model"].to_numpy() != "human").sum())

['human' 'mistral' 'llama' 'gpt' 'claude' 'falcon' 'palm' 'cohere' 'ada'
 'babbage' 'curie' 'davinci']
27373
46200


**Here we are balancing the dataset.**
In order to balance the dataset, I am just dropping the execessive values of the non-human to have the same number of datapoints in both the classes.(This is not recommended, as we are essentially losing a lot of data and learning.You can follow other techniques.)  

In [10]:
# Separate the 'Human' class
human_class = train_data[train_data['model'] == 'human']

# Separate the other classes
other_classes = train_data[train_data['model'] != 'human']

# Randomly sample 27373 rows from other classes
balanced_other_classes = other_classes.sample(n=len(human_class), random_state=42)

# Concatenate the 'Human' class and the balanced other classes
balanced_df = pd.concat([human_class, balanced_other_classes])

# Shuffle the dataframe
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Now 'balanced_df' contains the balanced dataset
balanced_df.drop(["source","RDizzl3_seven","model","prompt_name"],axis = 1, inplace=True)

**Displaying the balanced dataframe.**

In [11]:
balanced_df

,text,label
0,What do you think made the face on the side of...,0
1,I am strongly against this policy that the pri...,0
2,"Hi, I'm Kelvin and I go against this technolog...",0
3,Many people have come to believe that there is...,0
4,A life without cars sure sounds like a great i...,0
...,...,...
54741,"Have you ever dreamed of adventure, traveling ...",1
54742,"Important lesson,can tech you lessons, about ...",1
54743,Title: A Reflection on the Progress and Benefi...,1
54744,Texting & Driving\n\nUsing your phone while dr...,0


**Converting the dataframe to dataset format.**

In [12]:
balanced_dataset = Dataset.from_pandas(balanced_df)
balanced_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 54746
})

In [13]:
#@title Splitting the dataset into "Train(80%)" and "Test(20%)".
data = balanced_dataset.train_test_split(train_size=0.8, shuffle=True)
dataset = DatasetDict({
    'train': data['train'],
    'val': data['test']})
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 43796
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 10950
    })
})

**As one must know that the transformer based models accept tokenized data and not raw text.So we are tokenizing the raw data which will be given as input to the model(BERT).We are making use of the built-in tokenizer method in the transformer class(with appropraite parameters).**

In [16]:
#@title Tokenizing raw text
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding=True, max_length=512)

**Here we are defining the model we want to use for the classification i.e., BERT in my case. Instead of tokenizing each sentence individually by passing it through for loop,we can take advantage of the predefined method "map()"(as the Dataset is similar to Dictionary/can say same).**

In [15]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_dataset = dataset.map(function=tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/43796 [00:00<?, ? examples/s]

Map:   0%|          | 0/10950 [00:00<?, ? examples/s]

In [16]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 43796
    })
    val: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10950
    })
})

**Renaming the column "label" to "labels"**

In [17]:
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

**Dataloader to load the train data and the evalutation dataset for the model during the training process(with appropiate arguments.If unspecified the defaults will be taken.).**

In [18]:
train_loader = DataLoader(tokenized_dataset['train'], batch_size=16, shuffle=True)
eval_loader = DataLoader(tokenized_dataset['val'], batch_size=16,shuffle=True)

**Loading the model onto GPU if available, else we must use CPU.**

In [19]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print('cuda' if torch.cuda.is_available() else 'cpu')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cpu


**Inorder to evalute the model's performance we must use some metrics like accuracy in case of classification.So we are loading the metric "accuracy" from the package "evalute"."

In [14]:
metric = evaluate.load("accuracy")

**This function is used to calculate accuracy of the model during training.**

In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [22]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

**Intializing the model with the required hyperparameters.**

In [ ]:
#@title Intializing the model for training.
from transformers import TrainingArguments,Trainer
training_args = TrainingArguments(output_dir="/kaggle/working/run3", per_device_train_batch_size = 16, save_total_limit=2, disable_tqdm=False, load_best_model_at_end=True, num_train_epochs=5, evaluation_strategy="steps")
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['val'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
import wandb

# Disable wandb logging
wandb.init(mode="disabled")


**Training the model.**

In [ ]:
#@title Training the model
trainer.train()

# Testing the trained model.

In [9]:
test_data = pd.read_csv("/kaggle/input/daigt-proper-train-dataset/train_drcat_02.csv")
test_data

,essay_id,text,label,source,prompt,fold
0,E0B86027C5C5,"Okay, here is my essay:\n\nWorking or doing th...",1,llama2_chat,Task: \n\n1. Research the benefits of working ...,1
1,8B9EEF86DD66,"Dear Principal,\r\n\r\nI just want to let you ...",0,original_moth,Task: Research the effects of implementing a p...,2
2,116FB053BEC7,I believe they should change the voting to pop...,0,persuade_corpus,NaN,9
3,A72A4397F9F5,One example of an inspiring individual who has...,1,chat_gpt_moth,Task: Analyze examples of inspiring people who...,0
4,D90A159EDD27,"Dear Mr. Principal,\n\nI believe that students...",0,persuade_corpus,NaN,5
...,...,...,...,...,...,...
39780,F7341069C4A4,"""Oh man I didn't make the soccer team!"", yelle...",0,persuade_corpus,NaN,4
39781,AFE6E553DAC2,I believe that using this technology could be ...,0,persuade_corpus,NaN,2
39782,falcon_180b_v1_600,The Face on Mars is a fascinating phenomenon t...,1,falcon_180b_v1,You have read the article 'Unmasking the Face ...,2
39783,A5F84C104693,Texting & Driving\n\nUsing your phone while dr...,0,persuade_corpus,NaN,2


In [10]:
test_data.drop(["source","fold","essay_id","prompt"],axis = 1, inplace=True)

In [11]:
test_data['text']

0        Okay, here is my essay:\n\nWorking or doing th...
1        Dear Principal,\r\n\r\nI just want to let you ...
2        I believe they should change the voting to pop...
3        One example of an inspiring individual who has...
4        Dear Mr. Principal,\n\nI believe that students...
                               ...                        
39780    "Oh man I didn't make the soccer team!", yelle...
39781    I believe that using this technology could be ...
39782    The Face on Mars is a fascinating phenomenon t...
39783    Texting & Driving\n\nUsing your phone while dr...
39784    Dear Principal,\n\nI have been really good thi...
Name: text, Length: 39785, dtype: object

In [12]:
td = Dataset.from_pandas(test_data)
data1 = td.train_test_split(train_size=0.8, shuffle=True)
dataset1 = DatasetDict({
    'train': data1['train'],
    'val': data1['test']})
dataset1

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 31828
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 7957
    })
})

In [17]:
checkpoint1 = "/kaggle/input/outputftllm/results/run3/checkpoint-6500"
tokenizer = AutoTokenizer.from_pretrained(checkpoint1)
tokenized_dataset1 = dataset1.map(function=tokenize_function, batched=True)

Map:   0%|          | 0/31828 [00:00<?, ? examples/s]

Map:   0%|          | 0/7957 [00:00<?, ? examples/s]

In [18]:
tokenized_dataset1 = tokenized_dataset1.rename_column("label", "labels")

In [19]:
eval_loader1 = DataLoader(tokenized_dataset1['val'], batch_size=16,shuffle=True)

In [20]:
tokenized_dataset1['val']

Dataset({
    features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 7957
})

In [23]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint1, num_labels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print('cuda' if torch.cuda.is_available() else 'cpu')

test_dataset = tokenized_dataset1['val']

test_dataloader = DataLoader(test_dataset, batch_size=128)  # Adjust batch size as needed


# Define the Trainer object for evaluation
# Make sure to pass the trained model and the test DataLoader
trainer = Trainer(
    model=model,  # trained BERT model
    eval_dataset=test_dataset,  # test dataset
    compute_metrics=compute_metrics,  # evaluation metrics function
)


evaluation_results = trainer.evaluate()

accuracy = evaluation_results["eval_accuracy"]

print(accuracy)

cuda


0.9890662309915798


In [29]:
checkpoint = '/kaggle/input/outputftllm/results/run3/checkpoint-6500'
tokenizer = BertTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [30]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from transformers import AutoModelForSequenceClassification

def predict_single_sentence(sentence):
    # Tokenize and prepare the inputs
    inputs = tokenizer(sentence, padding=True, truncation=True, max_length=512, return_tensors="pt")

    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)

    # Process the model output (for a binary classification task)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    prediction = torch.argmax(probabilities, dim=1)

    # Assuming label 0 is "Human-generated" and label 1 is "AI-generated"
    labels = ["Human-generated", "AI-generated"]
    predicted_label = labels[prediction.item()]

    return predicted_label, probabilities[0].tolist()

# Example usage
sentence = 'Given that youre using AutoModelForSequenceClassification.from_pretrained with num_labels=2 and a specific torch_dtype, lets update the code snippet to reflect these specifics. Assuming checkpoint is the path or identifier for your model, and safetensors.get_torch_dtype() is a function call you have defined elsewhere to get a compatible torch.dtype, heres how you can adapt the example for predicting whether a single sentence is AI or human-generated'
predicted_label, probabilities = predict_single_sentence(sentence)
print(f"Predicted label: {predicted_label} with probabilities {probabilities}")

Predicted label: AI-generated with probabilities [6.547245902766008e-06, 0.9999934434890747]
